In [108]:
import pandas as pd
import unicodedata
import numpy as np

In [109]:
# Reading in Zagat ratings data, saved from scraping as pickle files
dc = pd.read_pickle('../dc.pkl') 
dc_prop = pd.read_pickle('../dc_prop.pkl') 
nyc = pd.read_pickle('../nyc.pkl') 
chicago = pd.read_pickle('../chicago.pkl') 
sf = pd.read_pickle('../sf.pkl') 

# read in Michelin star data
michelin = pd.read_pickle('../michelin.pkl')

# rename columns for easier merging
michelin.columns = ['address', 'title', 'stars']

In [110]:
# create a list of all of the datasets
datasets = [nyc, chicago, sf, dc, dc_prop, michelin]

In [111]:
for frame in datasets:
    print frame.shape

(2767, 13)
(1376, 13)
(2023, 13)
(1196, 13)
(690, 13)
(145, 3)


In [64]:
michelin.head(1)

,address,title,stars
0,"430 Hudson St., New York 10014",Piora,1


This next piece of syntax creates two functions that will be applied to restaurant names to make them easier to merge. One function is for text that needs to first be decoded into unicode, the other function is for text that is already unicode. The functions:
- remove accents
- put text in lowercase
- removes the word 'restaurant'
- remove '&' symbols (I found these didn't always match between datasets - Forest Inn Restaurant vs. Forest Inn & Restaurant
- replaces double spaces with a single space
- removes parentheses as (I found mismatches here too - dusek's (board beer) vs. dusek's board beer)
- strips extra leading or trailing spaces

I referenced this stackoverflow content in sorting out the accents: http://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string

In [112]:
#This version first converts to unicode:

def uni_convert_remove_accents(input_str):
    unicode_string = input_str.decode('utf-8')
    nfkd_form = unicodedata.normalize('NFKD', unicode_string)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.lower().replace('restaurant','').replace('&amp;', '').replace('&','').replace('  ', ' ').replace('(','').replace(')','').replace('the ','').strip()

#This version does not

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.lower().replace('restaurant','').replace('&amp;', '').replace('&','').replace('  ', ' ').replace('(','').replace(')','').replace('the ','').strip()


The next two steps apply the remove_accents function defined above to all of the datasets I'm working with. 

In [113]:
# next redefine the 'title' column of each dataset as the output of the remove_accents function
for frame in datasets:
    frame['title'] = frame['title'].apply(remove_accents)

In [114]:
# I also found that several New York restaurants had mismatching titles between Michelin and Zagat, so correcting those:
michelin.title = michelin.title.replace('ichimura', 'ichimura at brushstroke').replace('peter luger', 'peter luger steak house').replace('nomad','nomad hotel').replace('soto', 'soto japanese')

In [115]:
#testing...
michelin[michelin.title.str.contains('(ichimura)')]

C:\Users\elynchklarup\AppData\Local\Continuum\Anaconda2_take2\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


,address,title,stars
131,"30 Hudson St., New York 10013",ichimura at brushstroke,2


In [116]:
#check it out:
nyc.title.head(5)

0          daniel
1          bouley
2    le bernardin
3          per se
4        graffiti
Name: title, dtype: object

In [117]:
michelin.title.head()

0                  piora
1                   juni
2              take root
3    m. wells steakhouse
4                  cagen
Name: title, dtype: object

Now that my titles (restaurant names) are uniform across datasets, the next step is dividing the Michelin dataset by geography so that I can merge the Michelin data with my Zagat datasets. To do this, I'm filtering my aggregate Michelin dataset on zip code using regular expressions to create three sub-datasets:
- San Francisco: sf_michelin
- Chicago: chicago_michelin
- New York City: nyc_michelin
As shown here, the 'address' column of the Michelin dataset ends with each restaurant's zip code

In [118]:
michelin.head()

,address,title,stars
0,"430 Hudson St., New York 10014",piora,1
1,"12 E. 31st St., New York 10016",juni,1
2,"187 Sackett St., Brooklyn 11231",take root,1
3,"43-15 Crescent St., Long Island City 11101",m. wells steakhouse,1
4,"414 E. 9th St., New York 10009",cagen,1


In [119]:
# CA zip codes start with 9
sf_michelin = michelin[michelin['address'].str.contains('(9\d\d\d\d)$')]

C:\Users\elynchklarup\AppData\Local\Continuum\Anaconda2_take2\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


In [120]:
# Chicago zip codes start with 6
chicago_michelin = michelin[michelin['address'].str.contains('(6\d\d\d\d)$')]

C:\Users\elynchklarup\AppData\Local\Continuum\Anaconda2_take2\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


In [121]:
# NYC zip codes start with 0 or 1
nyc_michelin = michelin[michelin['address'].str.contains('((0|1)\d\d\d\d)$')]

C:\Users\elynchklarup\AppData\Local\Continuum\Anaconda2_take2\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


In [122]:
# To check that each Michelin restaurant was pulled into one dataset, I'll check dataset lengths. They match, as they should:
print len(nyc_michelin)+len(sf_michelin)+len(chicago_michelin)
print len(michelin)

145
145


My next step is merging the three city zagat datasets (sf, chicago, nyc) with my three Michelin datasets (sf_michelin, sf_chicago, sf_nyc). These will be merged on restaurant name (my title variable).

Another option would have been to merge based on geography. I have latitude and longitude from Zagat, and address from Michelin. I could use a package like GeoPy to 'georeference' the Michelin addresses (that is, extract latitude and longitude) and merge on lat & long. I'm using restaurant names instead because if I want to merge in additional restaurant data, I'm more likely to have restaurant name than address or lat/long. 

In [123]:
# San Francisco merge - first checking the sizes of my datasets:
print sf.shape
print sf_michelin.shape

(2023, 13)
(49, 3)


In [124]:
# Merge zagat and star information 
sf_zm = pd.merge(sf, sf_michelin, on = 'title', how = 'outer')

In [125]:
# Now looking at the shape of the merged dataset. If all of my Michelin restaurants had already been included in my Zagat
# dataset, the length of the dataset would be 2021 the same as the San Francisco zagat dataset. Since the length is 2026,
# it looks like 5 Michelin starred restaurants weren't included in Zagat:
sf_zm.shape

(2029, 15)

In [129]:
# Let's look at these restaurants more closely:
vars = ['title', 'address', 'stars']
sf_zm[vars].tail(6)

,title,address,stars
2023,nico,"3228 Sacramento St., San Francisco 94115",1.0
2024,wako,"211 Clement St., San Francisco 94118",1.0
2025,aziza,"5800 Geary Blvd., San Francisco 94121",1.0
2026,madrona manor,"1001 Westside Rd., Healdsburg 95448",1.0
2027,sushi yoshizumi,"325 E. 4th Ave., San Mateo 94401",1.0
2028,saison,"178 Townsend St., San Francisco 94107",3.0


Some Zagat searching reveals that yes, unfortunately, these restaurants don't seem to be Zagat-rated. Too bad. If I pull in Yelp data, I might use Yelp ratings to help impute Zagat ratings for these guys.
- Nico: no Zagat review
- Wako: no Zagat review
- Aziza: closed
- Madrona Manor: no Zagat review
- Sushi Yoshizumi: no Zagat review
- Saison: no Zagat review

Moving on - I'll repeat this merge for Chicago:

In [130]:
# And there is one restaurant without a Zagat rating
sf_zm[(sf_zm.stars >0) & (sf_zm.food ==0)]

,addr_city,cost,cuisine,latitude,longitude,neighborhood,open_table,price_level,title,url,food,decor,service,address,stars
2013,San Francisco,None,Sushi,37.7706528,-122.4027863,Potrero Hill,http://www.opentable.com/restaurant/profile/17...,VE,omakase,https://www.zagat.com/r/omakase-san-francisco,0.0,0.0,0.0,"665 Townsend St., San Francisco 94103",1.0


In [131]:
# check dataframe size
print chicago.shape
print chicago_michelin.shape

(1376, 13)
(21, 3)


In [132]:
# merge 
chicago_zm = pd.merge(chicago, chicago_michelin, on = 'title', how = 'outer')

In [133]:
#perfect match - number of rows stays the same
print chicago_zm.shape

(1376, 15)


In [134]:
# One restaurant with no Zagat rating -- prob need to omit, looks like this was recently overhauled
chicago_zm[(chicago_zm.stars >0) & (chicago_zm.food ==0)]

,addr_city,cost,cuisine,latitude,longitude,neighborhood,open_table,price_level,title,url,food,decor,service,address,stars
1024,Chicago,None,None,41.9134560,-87.6481628,Lincoln Park,,VE,alinea,https://www.zagat.com/r/alinea-chicago,0.0,0.0,0.0,"1723 N. Halsted St., Chicago 60614",3.0


And NYC: 

In [135]:
# check dataframe size
print nyc.shape
print nyc_michelin.shape

(2767, 13)
(75, 3)


In [136]:
# merge 
nyc_zm = pd.merge(nyc, nyc_michelin, on = 'title', how = 'outer')

In [137]:
# check shape - 4 Michelin restaurants without a Zagat match
nyc_zm.shape

(2771, 15)

In [145]:
# let's look further... 
# Juni has closed 
# Hirohisa is not Zagat rated
# Telepan has closed but has a Zagat review: https://www.zagat.com/r/telepan-new-york
# Luksus isn't Zagat reviewed

vars = ['title', 'address', 'food', 'decor', 'service', 'stars']
nyc_zm[vars].tail(4)

,title,address,food,decor,service,stars
2767,juni,"12 E. 31st St., New York 10016",NaN,NaN,NaN,1.0
2768,hirohisa,"73 Thompson St., New York 10012",NaN,NaN,NaN,1.0
2769,telepan,"72 W. 69th St., New York 10023",NaN,NaN,NaN,1.0
2770,luksus at trst,"615 Manhattan Ave., Brooklyn 11222",NaN,NaN,NaN,1.0


In [142]:
# This shows 5 restaurants that are on Zagat but have no rating
nyc_zm[(nyc_zm.stars >0) & (nyc_zm.food ==0)]

,addr_city,cost,cuisine,latitude,longitude,neighborhood,open_table,price_level,title,url,food,decor,service,address,stars
2134,New York,None,None,40.7449646,-73.9885483,NoMad,http://www.opentable.com/restaurant/profile/78...,None,nomad hotel,https://www.zagat.com/r/the-nomad-hotel-librar...,0.0,0.0,0.0,"1170 Broadway, New York 10001",1.0
2143,New York,None,Japanese,40.7481499,-73.9749146,Murray Hill,http://www.opentable.com/restaurant/profile/17...,VE,tempura matsui,https://www.zagat.com/r/tempura-matsui-new-york,0.0,0.0,0.0,"222 E. 39th St., New York 10158",1.0
2213,New York,None,Sushi,40.7224236,-74.0099182,TriBeCa,http://www.opentable.com/restaurant/profile/17...,E,sushi azabu,https://www.zagat.com/r/sushi-azabu-new-york,0.0,0.0,0.0,"428 Greenwich St., New York 10013",1.0
2361,New York,None,Japanese,40.7277374,-73.9842529,East Village,http://www.opentable.com/restaurant/profile/11...,E,cagen,https://www.zagat.com/r/cagen-new-york,0.0,0.0,0.0,"414 E. 9th St., New York 10009",1.0
2765,New York,None,French,40.7546196,-73.9827271,West 40s,http://www.opentable.com/restaurant/profile/18...,E,gabriel kreuther,https://www.zagat.com/r/gabriel-kreuther-new-york,0.0,0.0,0.0,"41 W. 42nd St., New York 10036",1.0


In [147]:
nyc[nyc.title.str.contains('(luger)')]

C:\Users\elynchklarup\AppData\Local\Continuum\Anaconda2_take2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


,addr_city,cost,cuisine,latitude,longitude,neighborhood,open_table,price_level,title,url,score5_food,score5_decor,score5_service
7,Brooklyn,88,Steakhouse,40.7098198,-73.9624634,Williamsburg,,VE,peter luger steak house,https://www.zagat.com/r/peter-luger-steak-hous...,4.7,3.9,4.3


In [151]:
nyc_zm[['title', 'cost', 'cuisine', 'open_table', 'price_level', 'food', 'decor', 'service', 'stars']].head(20)

,title,cost,cuisine,open_table,price_level,food,decor,service,stars
0,daniel,176,French,,VE,4.9,4.8,4.9,2.0
1,bouley,141,French,http://www.opentable.com/restaurant/profile/29...,VE,4.9,4.8,4.8,1.0
2,le bernardin,182,Seafood,http://www.opentable.com/restaurant/profile/25...,VE,4.9,4.8,4.9,3.0
3,per se,341,New American,http://www.opentable.com/restaurant/profile/27...,VE,4.8,4.7,4.8,3.0
4,graffiti,58,Asian,,E,4.8,4.3,4.7,NaN
5,jean-georges,166,French,http://www.opentable.com/restaurant/profile/31...,VE,4.8,4.8,4.8,3.0
6,eleven madison park,325,None,http://www.opentable.com/restaurant/profile/21...,VE,4.8,4.8,4.8,3.0
7,gotham bar and grill,91,American,http://www.opentable.com/restaurant/profile/62...,VE,4.8,4.6,4.7,1.0
8,peter luger steak house,88,Steakhouse,,VE,4.8,3.7,4.2,1.0
9,gramercy tavern,120,American,http://www.opentable.com/restaurant/profile/94...,VE,4.8,4.6,4.7,1.0


In [147]:
#saving clean datasets
dc.to_pickle('../dc_clean.pkl') 
nyc.to_pickle('../nyc_clean.pkl') 
chicago.to_pickle('../chicago_clean.pkl') 
sf.to_pickle('../sf_clean.pkl') 
dc_prop.to_pickle('../dc_prop_clean.pkl') 
michelin.to_pickle('../michelin_clean.pkl')

In [148]:
#saving merged datasets
nyc_zm.to_pickle('../nyc_zm.pkl')
sf_zm.to_pickle('../sf_zm.pkl')
chicago_zm.to_pickle('../chicago_zm.pkl')